In [38]:
import pandas as pd
from pathlib import Path

In [44]:
file_path = Path(r"/Users/akramchakrouni/Projects/time-series-forecasting-cluserting/data/chronos/AMZN.parquet")
data = pd.read_parquet(file_path)
ticker = file_path.stem

data

,Close,Open,High,Low,Volume
0,0.485588,0.492807,0.497787,0.488872,0.010589
1,0.504739,0.485328,0.504332,0.487903,0.007858
2,0.502242,0.505178,0.504856,0.503814,0.004311
3,0.488168,0.501896,0.501259,0.487937,0.004887
4,0.497874,0.487404,0.497444,0.491521,0.002950
...,...,...,...,...,...
5081,0.911870,0.914679,0.914947,0.916128,0.022267
5082,0.914209,0.911814,0.917783,0.916812,0.021832
5083,0.915858,0.914243,0.916287,0.919061,0.020931
5084,0.914271,0.915837,0.916786,0.919435,0.026428


In [17]:
from autogluon.timeseries import TimeSeriesDataFrame

ts_data = TimeSeriesDataFrame(data)
ts_data

Open        High         Low       Close  \
item_id timestamp                                                             
AAPL    2022-03-21 09:30:00  164.110001  165.589996  163.014999  163.225006   
        2022-03-21 10:30:00  163.220001  166.160004  163.220001  166.039902   
        2022-03-21 11:30:00  166.035004  166.350006  165.369995  165.615005   
        2022-03-21 12:30:00  165.619995  165.645004  163.160004  163.685501   
        2022-03-21 13:30:00  163.690002  164.789993  163.419998  164.679993   
...                                 ...         ...         ...         ...   
        2025-02-14 11:30:00  243.940002  244.529999  243.690002  243.815201   
        2025-02-14 12:30:00  243.824997  244.309998  243.339996  243.999893   
        2025-02-14 13:30:00  243.979996  244.539993  243.940002  244.149994   
        2025-02-14 14:30:00  244.139893  244.369995  243.910095  244.100006   
        2025-02-14 15:30:00  244.080002  245.410004  243.910004  244.570007   

                               Volume  
item_id timestamp                      
AAPL    2022-03-21 09:30:00  18295067  
        2022-03-21 10:30:00  15152948  
        2022-03-21 11:30:00   9968884  
        2022-03-21 12:30:00  10156181  
        2022-03-21 13:30:00   8501280  
...                               ...  
        2025-02-14 11:30:00   4178251  
        2025-02-14 12:30:00   2425988  
        2025-02-14 13:30:00   1978801  
        2025-02-14 14:30:00   2427820  
        2025-02-14 15:30:00   4882408  

[5086 rows x 5 columns]

In [27]:
file_path = r"/Users/akramchakrouni/Projects/time-series-forecasting-cluserting/data/chronos/AAPL.parquet"
data = pd.read_parquet(file_path)
data.rename(columns={"Date": "timestamp"}, inplace=True)

data

,Close,Open,High,Low,Volume
0,0.287312,0.293826,0.300777,0.288082,0.320497
1,0.308235,0.287211,0.304994,0.289602,0.265453
2,0.305077,0.308134,0.306400,0.305547,0.174637
3,0.290735,0.305050,0.301184,0.289158,0.177918
4,0.298127,0.290705,0.294858,0.291086,0.148927
...,...,...,...,...,...
5081,0.886318,0.887173,0.884869,0.886384,0.073196
5082,0.887691,0.886318,0.883241,0.883788,0.042499
5083,0.888806,0.887470,0.884943,0.888238,0.034665
5084,0.888435,0.888659,0.883685,0.888016,0.042531


In [20]:
a = pd.DataFrame(ts_data).to_csv("timeseries_data.csv", index=True)


AttributeError: 'NoneType' object has no attribute 'info'

In [43]:
from autogluon.timeseries import TimeSeriesPredictor

predictor = TimeSeriesPredictor(freq='1h', prediction_length=400)
predictor.fit(
    train_data=ts_data,
    presets="bolt_base",
    hyperparameters={
        "Chronos": {"model_path": "amazon/chronos-bolt-base"},
    },
)

Frequency '1h' stored as 'h'
Beginning AutoGluon training...
AutoGluon will save models to '/Users/akramchakrouni/Projects/time-series-forecasting-cluserting/notebooks/AutogluonModels/ag-20250214_185606'
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.11.11
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 24.2.0: Fri Dec  6 19:02:41 PST 2024; root:xnu-11215.61.5~2/RELEASE_ARM64_T6030
CPU Count:          11
GPU Count:          0
Memory Avail:       6.24 GB / 18.00 GB (34.7%)
Disk Space Avail:   377.24 GB / 460.43 GB (81.9%)
Setting presets to: bolt_base

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'freq': 'h',
 'hyperparameters': {'Chronos': {'model_path': 'amazon/chronos-bolt-base'}},
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 400,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every

In [44]:
forecast = predictor.predict(ts_data)
print(forecast.head())


data with frequency 'None' has been resampled to frequency 'h'.
Model not specified in predict, will default to the model with the best validation score: Chronos[amazon__chronos-bolt-base]


                                 mean       0.1       0.2       0.3       0.4  \
item_id timestamp                                                               
AAPL    2025-02-14 13:00:00  0.893889  0.882970  0.888185  0.890869  0.892642   
        2025-02-14 14:00:00  0.892471  0.876608  0.883882  0.887842  0.890387   
        2025-02-14 15:00:00  0.893507  0.874031  0.883028  0.887789  0.890990   
        2025-02-14 16:00:00  0.897408  0.874104  0.884431  0.890120  0.894120   
        2025-02-14 17:00:00  0.900429  0.873940  0.885616  0.892128  0.896698   

                                  0.5       0.6       0.7       0.8       0.9  
item_id timestamp                                                              
AAPL    2025-02-14 13:00:00  0.893889  0.895113  0.896587  0.898271  0.902394  
        2025-02-14 14:00:00  0.892471  0.894437  0.896744  0.899629  0.905584  
        2025-02-14 15:00:00  0.893507  0.896108  0.898883  0.902309  0.909636  
        2025-02-14 16:00:00  0.8

In [45]:
evaluation_results = predictor.evaluate(ts_data)
print(evaluation_results)


data with frequency 'None' has been resampled to frequency 'h'.
Model not specified in predict, will default to the model with the best validation score: Chronos[amazon__chronos-bolt-base]


{'WQL': -0.030171280564278553}


In [46]:
import matplotlib.pyplot as plt

# Suppose forecast has a similar structure to your original ts_data
plt.figure(figsize=(12, 6))
plt.plot(ts_data.query("item_id == 'AAPL'")['timestamp'], ts_data.query("item_id == 'AAPL'")['target'], label="Actual")
plt.plot(forecast.query("item_id == 'AAPL'")['timestamp'], forecast.query("item_id == 'AAPL'")['target'], label="Forecast")
plt.legend()
plt.xlabel("Timestamp")
plt.ylabel("Target")
plt.title("AAPL Forecast vs Actual")
plt.xticks(rotation=45)
plt.show()


KeyError: 'timestamp'

<Figure size 1200x600 with 0 Axes>

In [11]:
import pandas as pd

file_path = "/Users/akramchakrouni/Projects/time-series-forecasting-cluserting/data/chronos/PG.parquet"  # Adjust the path accordingly
df = pd.read_parquet(file_path)

# Check if "item_id" column exists
print("item_id" in df.columns)

# Display first few rows
print(df.head())


False
      Close      Open      High       Low    Volume
0  0.485739  0.483304  0.495127  0.481186  0.129831
1  0.475826  0.486261  0.487612  0.480839  0.070048
2  0.473217  0.476174  0.475287  0.470609  0.049314
3  0.478087  0.473217  0.476331  0.469135  0.062507
4  0.477217  0.478000  0.479464  0.481359  0.064993
